In [35]:
import sympy as sp

class Bond:
    def __init__(self, coupon, face, years):
        """
        SymPy-based Bond class (derivatives computed on demand).
        coupon : coupon amount per period (e.g., 100)
        face   : redemption / par value (e.g., 1000)
        years  : integer number of periods (e.g., 10)
        """
        # store numeric inputs
        self.coupon = float(coupon)
        self.face = float(face)
        self.years = int(years)
        
        # symbols
        self.y = sp.symbols('y')   # yield symbol
        self.x = sp.symbols('x')   # summation index
        
        # symbolic IV (price as function of y) with numeric coupon & face
        self.IV = sp.summation(self.coupon / (1 + self.y)**self.x, (self.x, 1, sp.Integer(self.years))) + self.face / (1 + self.y)**sp.Integer(self.years)

    # -------------------------
    # Numeric evaluation helpers
    # -------------------------
    def price(self, y_val):
        """Numeric price P(y)."""
        return float(self.IV.subs(self.y, float(y_val)))

    def dPdy(self, y_val):
        """Numeric first derivative dP/dy computed on demand."""
        dIV_dy = sp.diff(self.IV, self.y)
        return float(dIV_dy.subs(self.y, float(y_val)))

    def d2Pdy2(self, y_val):
        """Numeric second derivative d^2P/dy^2 computed on demand."""
        dIV_dy = sp.diff(self.IV, self.y)
        d2IV_dy2 = sp.diff(dIV_dy, self.y)
        return float(d2IV_dy2.subs(self.y, float(y_val)))

    # -------------------------
    # Duration & Convexity using derivatives
    # -------------------------
    def modified_duration(self, y_val):
        """Modified duration MD = - (1/P) * dP/dy"""
        P = self.price(y_val)
        dP = self.dPdy(y_val)
        return - dP / P

    def macaulay_duration(self, y_val):
        """Macaulay duration D = (1 + y) * MD"""
        MD = self.modified_duration(y_val)
        return (1.0 + float(y_val)) * MD

    def convexity(self, y_val):
        """Convexity = (1/P) * d^2P/dy^2"""
        P = self.price(y_val)
        d2P = self.d2Pdy2(y_val)
        return d2P / P

    # -------------------------
    # Linear & curved approx (for reference)
    # -------------------------
    def linear_approx_price(self, y_val, delta_y):
        """P(y) + dP/dy * delta_y"""
        P0 = self.price(y_val)
        dP = self.dPdy(y_val)
        return P0 + dP * delta_y

    def curved_approx_price(self, y_val, delta_y):
        """P(y) + dP/dy * dy + 0.5 * d2P/dy2 * dy^2"""
        P0 = self.price(y_val)
        dP = self.dPdy(y_val)
        d2P = self.d2Pdy2(y_val)
        return P0 + dP * delta_y + 0.5 * d2P * (delta_y**2)

    # -------------------------
    # Solve YTM numerically (nsolve)
    # -------------------------
    def solve_ytm(self, market_price, guess=0.05):
        """
        Solve IV(y) = market_price for y using sympy.nsolve.
        - market_price: numeric market price to match
        - guess: initial guess for y (decimal, e.g., 0.03 for 3%)
        Returns float(y) on success, raises ValueError on failure.
        """
        f = self.IV - float(market_price)
        try:
            y_solution = sp.nsolve(f, self.y, float(guess))
            return float(y_solution)
        except Exception as e:
            # nsolve can fail if guess is poor or function pathologic; surface a clear error
            raise ValueError(f"nsolve failed: {e}. Try a different initial guess or use a bracketing solver.") from e

    def __repr__(self):
        return f"Bond(coupon={self.coupon}, face={self.face}, years={self.years})"

# -------------------------
# Demo / usage
# -------------------------

b = Bond(coupon=100, face=1000, years=10)
print("Price", round(b.price(0.1), 2)) # 0.1 is market rate of interest i.e. 10%


print("Modified Duration:", round(b.modified_duration(0.1), 2))
print("Macaulay Duration:", round(b.macaulay_duration(0.1), 2))
print("Convexity:", round(b.convexity(0.1), 2))

# Solve YTM for given market price 

ytm = b.solve_ytm(1500) # YTM for the bond if market price is 1500   
print("Solved YTM (percent):", round(ytm * 100, 2), "%")


Price 1000.0
Modified Duration: 6.14
Macaulay Duration: 6.76
Convexity: 52.79
Solved YTM (percent): 3.87 %


In [ ]:
print("Price", round(b.price(0.1), 2)) # 0.1 is market rate of interest i.e. 10%